In [39]:
# %%writefile main_gg.py

import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import statsmodels.api as sm
# from scipy import stats
from datetime import datetime,timedelta
import calendar as c
import boto3

def isdir_s3(bucket, key):
    objs = list(bucket.objects.filter(Prefix=key))
#     print('I am the master_dir')
    return len(objs)

def costruzione_file_da_mensile(source, inizio, fine):
    bucketname = 'zus-prod-s3'
    s3 = boto3.resource('s3')
#     source = 'preprocessato/sistema/temperatura/epson/temperatura'
    my_bucket = s3.Bucket(bucketname)
    annomesi = pd.date_range(inizio, fine,freq='MS').strftime('%Y%m').tolist()
    df = pd.DataFrame()
    for am in annomesi:
        if isdir_s3(my_bucket,source + '/' +str(am))>0:
            df = df.append(pd.read_csv('s3://'+bucketname+'/'+source+'/'+str(am)+'/epson_best.csv'))
    return df

def compute_std_var(df_in,finestra_media_mobile_g,col):
    df = df_in.copy()
#     print(df_in)#.drop('provincia',axis=1).reset_index())
    days = int(np.floor(finestra_media_mobile_g/2))
    mds = sorted(df.monthday.unique())
    min_md = df[df['date']==min(df['date'])]['monthday'].ravel()[0]
    max_md = df[df['date']==max(df['date'])]['monthday'].ravel()[0]
    df['isleap'] = 0
    df['isleap'] = df.apply(lambda x: 1 if c.isleap(x['date'].year) else 0,axis=1)
    to_append = pd.DataFrame()
    for md in mds:
#         print(md)
        if md!='02-29':
            lista = pd.date_range(datetime.strptime(str(md), "%m-%d")-timedelta(days=days), datetime.strptime(str(md), "%m-%d")+timedelta(days=days)).strftime("%m-%d").tolist()
            var = df[df.monthday.isin(lista)][col].var(ddof=0)
#             print(df[df.monthday.isin(lista)])
            std = np.sqrt(var)
            to_append = to_append.append(pd.DataFrame({col+'_var':[var],col+'_std':[std]},index=[md]))
        else:
            lista = pd.date_range(datetime.strptime(str('02-22'), "%m-%d"), datetime.strptime(str('03-07'), "%m-%d")).strftime("%m-%d").tolist()
            lista = lista + ['02-29']
            var = df[(df.monthday.isin(lista)) & (df.isleap == 1)][col].var(ddof=0)
            std = np.sqrt(var)
            to_append = to_append.append(pd.DataFrame({col+'_var':[var],col+'_std':[std]},index=[md]))
    to_append.reset_index(inplace=True)
    to_append.rename(columns={'index':'monthday'},inplace=True)
    df = df.merge(to_append,on='monthday',how='left')
    return df

# def compute_std(df,finestra_media_mobile_g):
# #     df['monthday'] = df['date'].dt.month.astype('str').str.pad(2, side='left', fillchar='0') + '-' + df['date'].dt.day.astype('str').str.pad(2, side='left', fillchar='0')
#     return df.set_index(['dayofyear', 'year'], append=True).unstack().rolling(window=finestra_media_mobile_g, min_periods=1, center=True).var().stack().groupby('dayofyear')[['T_min_mean', 'T_max_mean','T_mean_m']].agg(lambda x: np.sqrt(np.mean(x)))    

# def compute_var(df,finestra_media_mobile_g):
#     return df.set_index(['dayofyear', 'year'], append=True).unstack().rolling(window=finestra_media_mobile_g, min_periods=1, center=True).var().stack().groupby('dayofyear')[['T_min_mean', 'T_max_mean','T_mean_m']].agg(lambda x: np.mean(x))

def compute_std_month(df,finestra_media_mobile_m):
#     print(df)
    df = df.set_index(['dayofyear', 'year'], append=True).unstack().rolling(window=finestra_media_mobile_m, min_periods=1, center=True).var().stack().groupby('dayofyear')[['T_min_sum', 'T_max_sum','T_mean_sum']].agg(lambda x: np.sqrt(np.mean(x)))  
    df = df.reset_index()
    df.rename(columns={'T_min_sum':'std_t_min', 'T_max_sum':'std_t_max','T_mean_sum':'std_t_mean'},inplace=True)
    return df
    
def compute_var_month(df,finestra_media_mobile_m):
    df = df.set_index(['dayofyear', 'year'], append=True).unstack().rolling(window=finestra_media_mobile_m, min_periods=1, center=True).var().stack().groupby('dayofyear')[['T_min_sum', 'T_max_sum','T_mean_sum']].agg(lambda x: np.mean(x))  
    df = df.reset_index()
    df.rename(columns={'T_min_sum':'var_t_min', 'T_max_sum':'var_t_max','T_mean_sum':'var_t_mean'},inplace=True)
    return df

def compute_std_month_oss(df,finestra_media_mobile_m):
    df = df.set_index(['dayofyear', 'year'],append=True).unstack().rolling(window=finestra_media_mobile_m, min_periods=1, center=True).var().stack().groupby('dayofyear')[['T_oss']].agg(lambda x: np.sqrt(np.mean(x)))  
    df = df.reset_index()
    df.rename(columns={'T_oss':'std_oss'},inplace=True)
    return df
    
def compute_var_month_oss(df,finestra_media_mobile_m):
    df = df.set_index(['dayofyear', 'year'],append=True).unstack().rolling(window=finestra_media_mobile_m, min_periods=1, center=True).var().stack().groupby('dayofyear')[['T_oss']].agg(lambda x: np.mean(x))  
    df = df.reset_index()
    df.rename(columns={'T_oss':'var_oss'},inplace=True)
    return df

def fit_harmonics(y, n_harmonics):
    n = y.shape[0]
    X = np.matrix([*[[np.sin(i*k/n*2*np.pi) for i in range(n)] for k in range(n_harmonics)], *[[np.cos(i*k/n*2*np.pi) for i in range(n)] for k in range(n_harmonics)]]).T
    X = sm.add_constant(X)

    ols_model = sm.OLS(y, X).fit()
    yhat = ols_model.predict(X)
    
    return yhat

# Calcolo percentili
def calcolo_percentili(df,column):
    if column=='mean':
        df['percentile_'+column+'_05_T_norm'] = df['T_'+column+'_m'] - 1.645 * df['std']
        df['percentile_'+column+'_95_T_norm'] = df['T_'+column+'_m'] + 1.645 * df['std']
        df['percentile_'+column+'_10_T_norm'] = df['T_'+column+'_m'] - 1.282 * df['std']
        df['percentile_'+column+'_90_T_norm'] = df['T_'+column+'_m'] + 1.282 * df['std']
        df['percentile_'+column+'_25_T_norm'] = df['T_'+column+'_m'] - 0.675 * df['std']
        df['percentile_'+column+'_75_T_norm'] = df['T_'+column+'_m'] + 0.675 * df['std']
    else:
        df['percentile_'+column+'_05_T_norm'] = df['T_'+column+'_mean'] - 1.645 * df['std']
        df['percentile_'+column+'_95_T_norm'] = df['T_'+column+'_mean'] + 1.645 * df['std']
        df['percentile_'+column+'_10_T_norm'] = df['T_'+column+'_mean'] - 1.282 * df['std']
        df['percentile_'+column+'_90_T_norm'] = df['T_'+column+'_mean'] + 1.282 * df['std']
        df['percentile_'+column+'_25_T_norm'] = df['T_'+column+'_mean'] - 0.675 * df['std']
        df['percentile_'+column+'_75_T_norm'] = df['T_'+column+'_mean'] + 0.675 * df['std']
    return df

def calcolo_percentili_yhat(df,column):
    df['percentile_'+column+'_05_yhat'] = df['yhat_'+column] - 1.645 * df['std']
    df['percentile_'+column+'_95_yhat'] = df['yhat_'+column] + 1.645 * df['std']
    df['percentile_'+column+'_10_yhat'] = df['yhat_'+column] - 1.282 * df['std']
    df['percentile_'+column+'_90_yhat'] = df['yhat_'+column] + 1.282 * df['std']
    df['percentile_'+column+'_25_yhat'] = df['yhat_'+column] - 0.675 * df['std']
    df['percentile_'+column+'_75_yhat'] = df['yhat_'+column] + 0.675 * df['std']
    return df

def calcolo_percentili_oss_yhat(df):
    df['percentile_05_yhat'] = df['yhat'] - 1.645 * df['std']
    df['percentile_95_yhat'] = df['yhat'] + 1.645 * df['std']
    df['percentile_10_yhat'] = df['yhat'] - 1.282 * df['std']
    df['percentile_90_yhat'] = df['yhat'] + 1.282 * df['std']
    df['percentile_25_yhat'] = df['yhat'] - 0.675 * df['std']
    df['percentile_75_yhat'] = df['yhat'] + 0.675 * df['std']
    return df

def calcolo_percentili_oss_T_oss(df):
    df['percentile_05_T_oss'] = df['T_oss'] - 1.645 * df['std']
    df['percentile_95_T_oss'] = df['T_oss'] + 1.645 * df['std']
    df['percentile_10_T_oss'] = df['T_oss'] - 1.282 * df['std']
    df['percentile_90_T_oss'] = df['T_oss'] + 1.282 * df['std']
    df['percentile_25_T_oss'] = df['T_oss'] - 0.675 * df['std']
    df['percentile_75_T_oss'] = df['T_oss'] + 0.675 * df['std']
    return df

def intervalli_confidenza(df,column):
    df['c_i_95_'+column+'_l'] = df[column] - 1.96 * df['std']/np.sqrt(df['n_osservazioni'])
    df['c_i_95_'+column+'_h'] = df[column] + 1.96 * df['std']/np.sqrt(df['n_osservazioni'])
    return df

#def lettura_temp_norm(file_t_norm,inizio,fine,finestra_media_mobile_g):
def lettura_temp_norm(df,inizio,fine,finestra_media_mobile_g,tipo):

#     df = pd.read_csv(file_t_norm)#,encoding='utf-16')
    #df = pd.read_csv(file_t_norm)#,encoding='utf-16')
    
    #df.rename(columns={'TIME - Date':'date','Codice Provincia':'provincia','Temperature Min':'temp_min','Temperature Max':'temp_max'},inplace=True)


    #df = costruzione_file_da_mensile(file_t_norm,inizio,fine)
#     if t.lower() = 'prov':
#         tipo = 'PROVINCIA'
#     else:
#         tipo = 'OSSERVATORIO'
    df = df[['DATA',tipo.upper(),'TEMPERATURA_MAX','TEMPERATURA_MIN']]
    
    df.rename(columns={'DATA':'date',tipo.upper():tipo.lower(),'TEMPERATURA_MIN':'temp_min','TEMPERATURA_MAX':'temp_max'},inplace=True)
    tipo = tipo.lower()
    df = df[~df[tipo].isnull()]
    
    df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')
    df = df[(df['date']>=inizio) & (df['date']<=fine)]
    df['T_mean'] = df[['temp_min', 'temp_max']].mean(axis=1)
#     df.drop('Desc_Provincia',axis=1,inplace=True)

    df = df.assign(
        dayofyear=lambda x: (x.date).dt.dayofyear, # + pd.DateOffset(days=92)
        year=lambda x: (x.date).dt.year, # + pd.DateOffset(days=92)
    )
#     df_con = df.assign(
#         dayofyear=lambda x: (x.date + pd.DateOffset(days=92)).dt.dayofyear, # + pd.DateOffset(days=92)
#         year=lambda x: (x.date + pd.DateOffset(days=92)).dt.year, # + pd.DateOffset(days=92)
#     )

#     QUA SOTTO METTERE <=9 PER OFFSET=92, SENZA OFFSET METTERE <=12
    df['dayofyear'] = df.apply(lambda x: x['dayofyear']+1 if (c.isleap(x['year'])==False and x['date'].month>=3 and x['date'].month<=12) else x['dayofyear'],axis=1)
    
    df['monthyear'] = (df.date.dt.month).astype('str').str.pad(2, side='left', fillchar='0')+(df.year).astype('str')
#     df_con['monthyear'] = (df_con.date.dt.month).astype('str').str.pad(2, side='left', fillchar='0')+(df_con.year).astype('str')
    
#     df.to_csv('df_senza_92.csv',index=False)
#     df_con.to_csv('df_con_92.csv',index=False)
#     ciao
#     Raggruppo per provincia e monthyear
    tmp = (df[['monthyear','date',tipo]].groupby(['monthyear',tipo]).count()).reset_index()
    tmp['monthyear'] = tmp['monthyear'].astype('str')
    tmp['monthyear'] = tmp['monthyear'].str.slice(stop=2)
    tmp.rename(columns={'date':'n_anni','monthyear':'month'},inplace=True)
    tmp['n_anni'] = 1
    
    tmp = tmp.groupby(['month',tipo]).count().reset_index()
    
    df = df.merge(tmp,on=[tipo],how='left')
    
    df['n_osservazioni'] = df['n_anni']*finestra_media_mobile_g

    df.drop(['n_anni','month'],axis=1,inplace=True)
    df.drop_duplicates(inplace=True)
    df['monthday'] = df['date'].dt.month.astype('str').str.pad(2, side='left', fillchar='0') + '-' + df['date'].dt.day.astype('str').str.pad(2, side='left', fillchar='0')   
    return df

def lettura_osservatorio(df,inizio,fine,finestra_media_mobile_g):
#     df_oss = pd.read_csv(file_oss)
    #df_oss.rename(columns={'Cd Oss':'codice_oss','T Oss':'T_oss','Data':'date'},inplace=True)
    
    df_oss['date'] = df_oss['date'].astype('str')
    df_oss['date'] = pd.to_datetime(df_oss['date'],format='%Y-%m-%d', exact=False)
    df_oss = df_oss[(df_oss['date']>=inizio) & (df_oss['date']<=fine)]
    df_oss = df_oss[~df_oss['codice_oss'].isnull()]

    df['T_mean'] = df[['temp_min', 'temp_max']].mean(axis=1)
    df.drop('Desc_Provincia',axis=1,inplace=True)
    
    df_oss = df_oss.assign(
        dayofyear=lambda x: (x.date).dt.dayofyear, # + pd.DateOffset(days=92)
        year=lambda x: (x.date).dt.year, # + pd.DateOffset(days=92)
    )
    
    df_oss['dayofyear'] = df_oss.apply(lambda x: x['dayofyear']+1 if (c.isleap(x['year'])==False and x['date'].month>=3 and x['date'].month<=12) else x['dayofyear'],axis=1)
    
    df_oss.drop('Oss',axis=1,inplace=True)
    df_oss['monthyear'] = (df_oss.date.dt.month).astype('str').str.pad(2, side='left', fillchar='0')+(df_oss.date.dt.year).astype('str')
    tmp = (df_oss[['monthyear','date','codice_oss']].groupby(['monthyear','codice_oss']).count()).reset_index()
    tmp.rename(columns={'date':'n_anni'},inplace=True) #,'monthyear':'month'
    tmp['n_anni'] = 1
    tmp['monthyear'] = tmp['monthyear'].astype('str')
    tmp['month'] = tmp['monthyear'].str.slice(stop=2)

    tmp = tmp.groupby(['month','codice_oss']).count().reset_index()
    tmp.drop('monthyear',axis=1,inplace=True)
    df_oss['month'] = df_oss['monthyear'].str.slice(stop=2)
    df_oss = df_oss.merge(tmp,on=['codice_oss','month'],how='left')
    
    df_oss['n_osservazioni'] = df_oss['n_anni']*finestra_media_mobile_g
    
    df_oss.drop(['n_anni'],axis=1,inplace=True)
    df_oss['monthday'] = df_oss['date'].dt.month.astype('str').str.pad(2, side='left', fillchar='0') + '-' + df_oss['date'].dt.day.astype('str').str.pad(2, side='left', fillchar='0')
    return df_oss

def calcolo_temp_norm_gg(file_t_norm,inizio,fine,finestra_media_mobile_g,t):
    if t.lower()=='prov':
        tipo = 'PROVINCIA'
    else:
        tipo='OSSERVATORIO'
#     Calcolo temperature normali giornaliere
    print("Lettura file temperature per "+ str(tipo))
    df = lettura_temp_norm(file_t_norm,inizio,fine,finestra_media_mobile_g,tipo)
    tipo = tipo.lower()
    print("Numero "+tipo+" presenti: ",df[tipo].nunique())
    
    print("Calcolo deviazione standard.")
    df_max = df.groupby(tipo).apply(compute_std_var, finestra_media_mobile_g=finestra_media_mobile_g, col='temp_max')[[tipo,'date','temp_max_var','temp_max_std']]
    df_min = df.groupby(tipo).apply(compute_std_var, finestra_media_mobile_g=finestra_media_mobile_g, col='temp_min')[[tipo,'date','temp_min_var','temp_min_std']]
    df_mean = df.groupby(tipo).apply(compute_std_var, finestra_media_mobile_g=finestra_media_mobile_g, col='T_mean')[[tipo,'date','T_mean_var','T_mean_std']]

    df_max.drop(tipo,axis=1,inplace=True)
    df_min.drop(tipo,axis=1,inplace=True)
    df_mean.drop(tipo,axis=1,inplace=True)
    
    df_max.reset_index(inplace=True)
    df_min.reset_index(inplace=True)
    df_mean.reset_index(inplace=True)
    
    df = df.merge(df_max, on=[tipo,'date'], how='left')
    df = df.merge(df_min, on=[tipo,'date'], how='left')
    df = df.merge(df_mean, on=[tipo,'date'], how='left')
    
    df.rename(columns={'temp_max':'T_max_mean','temp_min':'T_min_mean','T_mean':'T_mean_m'},inplace=True)
    
#     df.to_csv("df_senza_92.csv",index=False)
    
    max_min_gg = df.groupby(['dayofyear',tipo]).agg({'T_min_mean':['min'],'T_max_mean':['max']})
    max_min_gg.columns = max_min_gg.columns.droplevel(0)
    max_min_gg = max_min_gg.reset_index()
    max_min_gg.rename(columns={'min':'T_min_abs','max':'T_max_abs'},inplace=True)
    
    print("Calcolo media giornaliera.")
    temp_norm = df.groupby(['dayofyear', tipo])[['T_min_mean', 'T_max_mean','T_mean_m', 'temp_max_std', 'temp_max_var', 'temp_min_std', 'temp_min_var', 'T_mean_std', 'T_mean_var']].mean().unstack().reindex(range(367)).interpolate(method='linear').stack()#.reset_index() #,'T_mean'
    
    province = temp_norm.reset_index([tipo])[tipo].unique()
    finale = pd.DataFrame()

    print("Inizio procedura di fit con armoniche di sesto grado.")
#     Fit armoniche di sesto grado per curve T_min, T_max e T_mean
    for p in province:
        temp_p = temp_norm.query('{} == {}'.format(tipo,p)).reset_index([tipo,'dayofyear'])#.assign(std_mean=lambda x: x['std'], iv_l=lambda x: (x['mean'] - 2*x['std'].rolling(window=15, center=True).mean()), iv_h=lambda x: (x['mean'] + 2*x['std'].rolling(window=15, center=True).mean()))
        ###########################
#         temp_p = temp_p.merge(tmp,on='')
        ###########################
        y_min = temp_p['T_min_mean'].values.ravel()
        y_max = temp_p['T_max_mean'].values.ravel()
        y_mean = temp_p['T_mean_m'].values.ravel()

        x = temp_p.index
        
        yhat_min = fit_harmonics(y_min, 6)
        yhat_max = fit_harmonics(y_max, 6)
        yhat_mean = fit_harmonics(y_mean, 6)

        temp_p['yhat_min'] = yhat_min
        temp_p['yhat_max'] = yhat_max
        temp_p['yhat_mean'] = yhat_mean
        
#         if p==1:
# #             temp_p.to_csv('temp_p_con_92.csv',index=False)
#             print(y_mean)

        finale = finale.append(temp_p)
        
#     Calcolo gradi giorno

#################
#     ciao
#################
    print("Conversione in gradi giorno.")
    finale['gradi_giorno_yhat_mean'] = 0
    finale['gradi_giorno_yhat_mean'] = finale.apply(lambda x: max(0,18-x['yhat_mean']),axis=1)
    finale.reset_index(inplace=True)
    
    finale['gradi_giorno_T_mean'] = 0
    finale['gradi_giorno_T_mean'] = finale.apply(lambda x: max(0,18-x['T_mean_m']),axis=1)
    finale.reset_index(inplace=True)
    
    selezione = df[[tipo,'n_osservazioni']]
    selezione = selezione.groupby(tipo).apply(highest_number)
    selezione.drop_duplicates(inplace=True)
    finale = finale.merge(selezione,on=[tipo],how='left')

    temp_max = temp_norm[['T_max_mean', 'temp_max_std', 'temp_max_var']].rename(columns={'temp_max_std': 'std', 'temp_max_var': 'var'}).reset_index()
    temp_min = temp_norm[['T_min_mean', 'temp_min_std', 'temp_min_var']].rename(columns={'temp_min_std': 'std', 'temp_min_var': 'var'}).reset_index()
    temp_mean = temp_norm[['T_mean_m', 'T_mean_std', 'T_mean_var']].rename(columns={'T_mean_std': 'std', 'T_mean_var': 'var'}).reset_index()

    temp_max = temp_max.merge(finale[['dayofyear',tipo,'yhat_max','n_osservazioni']],on=['dayofyear',tipo],how='left')
    temp_min = temp_min.merge(finale[['dayofyear',tipo,'yhat_min','n_osservazioni']],on=['dayofyear',tipo],how='left')
    temp_mean = temp_mean.merge(finale[['dayofyear',tipo,'yhat_mean','gradi_giorno_yhat_mean','gradi_giorno_T_mean','n_osservazioni']],on=['dayofyear',tipo],how='left')

    print("Calcolo percentili per curve di minimo, massimo e media.")
    temp_max = calcolo_percentili(temp_max,'max')
    temp_min = calcolo_percentili(temp_min,'min')
    temp_mean = calcolo_percentili(temp_mean,'mean')

    temp_max = calcolo_percentili_yhat(temp_max,'max')
    temp_min = calcolo_percentili_yhat(temp_min,'min')
    temp_mean = calcolo_percentili_yhat(temp_mean,'mean')

    print("Calcolo intervalli di confidenza per curve di minimo, massimo e media.")
    temp_max = intervalli_confidenza(temp_max,'T_max_mean')
    temp_min = intervalli_confidenza(temp_min,'T_min_mean')
    temp_mean = intervalli_confidenza(temp_mean,'T_mean_m')
    
    temp_max = intervalli_confidenza(temp_max,'yhat_max')
    temp_min = intervalli_confidenza(temp_min,'yhat_min')
    temp_mean = intervalli_confidenza(temp_mean,'yhat_mean')
    
    prv = pd.DataFrame()
    prv['dayofyear'] = range(1,367)
    prv['monthday'] = pd.date_range('2020-01-01','2020-12-31').astype('str').str.slice(start=5) #'2019-10-01','2020-09-30' for offset=92!
    
    temp_max = temp_max.merge(prv,on='dayofyear',how='left')
    temp_min = temp_min.merge(prv,on='dayofyear',how='left')
    temp_mean = temp_mean.merge(prv,on='dayofyear',how='left')

    temp_max['mese'] = temp_max['monthday'].str.slice(stop=2).astype('int')
    temp_max['giorno'] = temp_max['monthday'].str.slice(start=-2).astype('int')
    
    temp_min['mese'] = temp_min['monthday'].str.slice(stop=2).astype('int')
    temp_min['giorno'] = temp_min['monthday'].str.slice(start=-2).astype('int')
    
    temp_mean['mese'] = temp_mean['monthday'].str.slice(stop=2).astype('int')
    temp_mean['giorno'] = temp_mean['monthday'].str.slice(start=-2).astype('int')

    temp_max = temp_max.merge(max_min_gg,on=['dayofyear',tipo],how='left')
    temp_min = temp_min.merge(max_min_gg,on=['dayofyear',tipo],how='left')
    temp_mean = temp_mean.merge(max_min_gg,on=['dayofyear',tipo],how='left')

    temp_max = temp_max[['dayofyear','mese','giorno',tipo,'T_max_mean','T_min_abs','T_max_abs','std','var','yhat_max','percentile_max_10_T_norm','percentile_max_90_T_norm','percentile_max_10_yhat','percentile_max_90_yhat','c_i_95_T_max_mean_l','c_i_95_T_max_mean_h','c_i_95_yhat_max_l','c_i_95_yhat_max_h']]
    temp_min = temp_min[['dayofyear','mese','giorno',tipo,'T_min_mean','T_min_abs','T_max_abs','std','var','yhat_min','percentile_min_10_T_norm','percentile_min_90_T_norm','percentile_min_10_yhat','percentile_min_90_yhat','c_i_95_T_min_mean_l','c_i_95_T_min_mean_h','c_i_95_yhat_min_l','c_i_95_yhat_min_h']]
    temp_mean = temp_mean[['dayofyear','mese','giorno',tipo,'T_mean_m','T_min_abs','T_max_abs','gradi_giorno_yhat_mean','gradi_giorno_T_mean','std','var','yhat_mean','percentile_mean_10_T_norm','percentile_mean_90_T_norm','percentile_mean_10_yhat','percentile_mean_90_yhat','c_i_95_T_mean_m_l','c_i_95_T_mean_m_h','c_i_95_yhat_mean_l','c_i_95_yhat_mean_h']]
    
#     temp_max.rename(columns={'T_min_abs':'t_min_abs','T_max_abs':'t_max_abs','T_max_mean':'t_mean','yhat_max':'yhat','percentile_max_10_T_norm':'percentile_10_t_mean','percentile_max_90_T_norm':'percentile_90_t_mean','percentile_max_10_yhat':'percentile_10_yhat','percentile_max_90_yhat':'percentile_90_yhat','c_i_95_T_max_mean_l':'c_i_95_t_mean_l','c_i_95_T_max_mean_h':'c_i_95_t_mean_u','c_i_95_yhat_max_l':'c_i_95_yhat_l','c_i_95_yhat_max_h':'c_i_95_yhat_u'},inplace=True)
#     temp_min.rename(columns={'T_min_abs':'t_min_abs','T_max_abs':'t_max_abs','T_min_mean':'t_mean','yhat_min':'yhat','percentile_min_10_T_norm':'percentile_10_t_mean','percentile_min_90_T_norm':'percentile_90_t_mean','percentile_min_10_yhat':'percentile_10_yhat','percentile_min_90_yhat':'percentile_90_yhat','c_i_95_T_min_mean_l':'c_i_95_t_mean_l','c_i_95_T_min_mean_h':'c_i_95_t_mean_u','c_i_95_yhat_min_l':'c_i_95_yhat_l','c_i_95_yhat_min_h':'c_i_95_yhat_u'},inplace=True)
#     temp_mean.rename(columns={'T_min_abs':'t_min_abs','T_max_abs':'t_max_abs','T_mean_m':'t_mean','yhat_mean':'yhat','percentile_mean_10_T_norm':'percentile_10_t_mean','percentile_mean_90_T_norm':'percentile_90_t_mean','percentile_mean_10_yhat':'percentile_10_yhat','percentile_mean_90_yhat':'percentile_90_yhat','c_i_95_T_mean_m_l':'c_i_95_t_mean_l','c_i_95_T_mean_m_h':'c_i_95_t_mean_u','c_i_95_yhat_mean_l':'c_i_95_yhat_l','c_i_95_yhat_mean_h':'c_i_95_yhat_u'},inplace=True)

    return temp_max,temp_min,temp_mean


def highest_number(df):
    df['n_osservazioni'] = df.n_osservazioni.max()
    return df

def calcolo_oss_gg(file_oss,inizio,fine,finestra_media_mobile_g):

    print("Lettura file temperature per osservatorio.")
    df_oss = lettura_osservatorio(file_oss,inizio,fine,finestra_media_mobile_g)
    
    print("Numero osservatori presenti: ",df_oss['codice_oss'].nunique())

    df_oss = df_oss.assign(
        dayofyear=lambda x: (x.date).dt.dayofyear, # + pd.DateOffset(days=92)
        year=lambda x: (x.date).dt.year, # + pd.DateOffset(days=92)
    )
    
    print("Calcolo deviazione standard.")
    df_oss_std_var = df_oss.groupby('codice_oss').apply(compute_std_var, finestra_media_mobile_g=finestra_media_mobile_g, col='T_oss')[['codice_oss','date','T_oss_var','T_oss_std']]
    
    df_oss_std_var.drop('codice_oss',axis=1,inplace=True)

    df_oss_std_var.reset_index(inplace=True)
    df_oss = df_oss.merge(df_oss_std_var,on=['codice_oss','date'],how='left')

    print("Calcolo media giornaliera.")
    temp_oss = df_oss.groupby(['dayofyear', 'codice_oss'])[['T_oss','T_oss_var','T_oss_std']].mean().unstack().reindex(range(367)).interpolate(method='linear').stack() #,'T_mean'

    osservatori = temp_oss.reset_index(['codice_oss']).codice_oss.unique()
    finale_oss = pd.DataFrame()

    print("Inizio procedura di fit con armoniche di sesto grado.")
    for o in osservatori:
        temp_o = temp_oss.query('codice_oss == {}'.format(o)).reset_index(['codice_oss'])#.assign(std_mean=lambda x: x['std'], iv_l=lambda x: (x['mean'] - 2*x['std'].rolling(window=15, center=True).mean()), iv_h=lambda x: (x['mean'] + 2*x['std'].rolling(window=15, center=True).mean()))

        y = temp_o['T_oss'].values.ravel()
        x = temp_o.index
        
        yhat = fit_harmonics(y, 6)

        temp_o['yhat'] = yhat

        finale_oss = finale_oss.append(temp_o)
        
    print("Conversione in gradi giorno.")
    finale_oss['gradi_giorno_yhat'] = 0
    finale_oss['gradi_giorno_yhat'] = finale_oss.apply(lambda x: max(0,18-x['yhat']),axis=1)
    finale_oss.reset_index(inplace=True)#[finale['T_mean_m']>=18]

    finale_oss['gradi_giorno_T_oss'] = 0
    finale_oss['gradi_giorno_T_oss'] = finale_oss.apply(lambda x: max(0,18-x['T_oss']),axis=1)
    finale_oss.reset_index(inplace=True)#[finale['T_mean_m']>=18]
    
    selezione = df_oss[['codice_oss','n_osservazioni']]
    selezione = selezione.groupby('codice_oss').apply(highest_number)
    selezione.drop_duplicates(inplace=True)
        
    finale_oss = finale_oss.merge(selezione,on=['codice_oss'],how='left')

    temp_oss = temp_oss.reset_index()
    temp_oss.drop_duplicates(inplace=True)
    temp_oss = temp_oss.set_index(['dayofyear','codice_oss'])

    temp_oss_sv = temp_oss[['T_oss', 'T_oss_std', 'T_oss_var']].rename(columns={'T_oss_std': 'std', 'T_oss_var': 'var'}).reset_index()
    temp_oss_sv = temp_oss_sv.merge(finale_oss[['dayofyear','codice_oss','yhat','gradi_giorno_T_oss','gradi_giorno_yhat','n_osservazioni']],on=['dayofyear','codice_oss'],how='left')

    temp_oss_sv.drop_duplicates(inplace=True)

    print("Calcolo percentili per curve di minimo, massimo e media.")
    temp_oss_sv = calcolo_percentili_oss_yhat(temp_oss_sv)
    temp_oss_sv = calcolo_percentili_oss_T_oss(temp_oss_sv)
    
    print("Calcolo intervalli di confidenza per curve di minimo, massimo e media.")
    temp_oss_sv_T_oss = intervalli_confidenza(temp_oss_sv,'T_oss')
    temp_oss_sv_yhat = intervalli_confidenza(temp_oss_sv,'yhat')
    
    prv = pd.DataFrame()
    prv['dayofyear'] = range(1,367)
    prv['monthday'] = pd.date_range('2020-01-01','2020-12-31').astype('str').str.slice(start=5)

    temp_oss_sv_yhat = temp_oss_sv_yhat.merge(prv,on='dayofyear',how='left')
    
    temp_oss_sv_yhat['mese'] = temp_oss_sv_yhat['monthday'].str.slice(stop=2).astype('int')
    temp_oss_sv_yhat['giorno'] = temp_oss_sv_yhat['monthday'].str.slice(start=-2).astype('int')
    
    temp_oss_sv_yhat = temp_oss_sv_yhat[['dayofyear','mese','giorno','codice_oss','gradi_giorno_yhat','gradi_giorno_T_oss','T_oss','std','var','yhat','percentile_10_yhat','percentile_90_yhat','percentile_10_T_oss','percentile_90_T_oss','c_i_95_T_oss_l','c_i_95_T_oss_h','c_i_95_yhat_l','c_i_95_yhat_h']]
    
    return temp_oss_sv_yhat

def main_gg(file_input,inizio,fine,finestra_media_mobile_g,path_to_output,tipo):
    
    print('Funzione dedicata al calcolo delle temperature giornaliere e mensili.')
    print('\n')
    
    my_bucket = 'zus-prod-s3'
    
    finestra_media_mobile_g = int(finestra_media_mobile_g)
    
    inizio_dt = pd.to_datetime(inizio,format="%Y-%m-%d")
    fine_dt = pd.to_datetime(fine,format="%Y-%m-%d")
    
    now = str(datetime.now())[0:-7].replace('-','').replace(' ','').replace(':','')
    
    file_name_out = inizio.replace('-','') + '_' + fine.replace('-','') + '_' + str(datetime.now())[0:-7].replace('-','').replace(' ','').replace(':','')
        
    #df = pd.read_csv('s3://'+ my_bucket +'/'+file_input)#,encoding='utf-16')
    df = costruzione_file_da_mensile(file_input,inizio,fine)
    
    #if 'Codice Provincia' in df.columns:
    #if 'PROVINCIA' in df.columns:
    #if tipo.upper() == 'PROVINCIA':
    print('Calcolo temperatura normale giornaliera per '+str(tipo)+' con funzioni armoniche di 6^ grado')
#         temp_max_norm, temp_min_norm, temp_mean_norm = calcolo_temp_norm_gg('s3://'+ my_bucket +'/'+file_input,inizio_dt,fine_dt,finestra_media_mobile_g)
    temp_max_norm, temp_min_norm, temp_mean_norm = calcolo_temp_norm_gg(df,inizio_dt,fine_dt,finestra_media_mobile_g,tipo)

#         temp_max_norm.drop('dayofyear',inplace=True,axis=1)
#         temp_min_norm.drop('dayofyear',inplace=True,axis=1)
#         temp_mean_norm.drop('dayofyear',inplace=True,axis=1)

    temp_max_norm.to_csv('s3://'+ my_bucket +'/'+path_to_output+'max_d_'+tipo+'/'+file_name_out+'/max_d_'+tipo+'.csv',index=False)
    idrun_max = path_to_output+'/max_d_'+tipo+'/'+file_name_out+'/max_d_'+tipo+'.csv'
    metadatati_max = pd.DataFrame(data={'CATEGORIA':['TEMPERATURA_NORM']*4,'FLUSSO':['MAX_D_'+tipo.upper()]*4,'TIMESTAMP':[now]*4,'ID':['TEMPERATURA_NORM_MAX_D_'+tipo.upper()+'_'+now]*4,'PATH':[idrun_max]*4,'NOME_PARAMETRO':['INIZIO_INIZIO','INIZIO_FINE','FINESTRA_MEDIA_MOBILE_G','PATH_TO_OUTPUT'],'VALORE_PARAMETRO':[inizio,fine,finestra_media_mobile_g,path_to_output]})
    metadatati_max.to_csv('s3://'+ my_bucket +'/metadati/sistema/temperatura_norm/zeus/metadati/max_d_'+tipo+'/'+file_name_out+'/metadati.csv',index=False)
    #metadatati_max.to_csv('./metadati/metadati_prov_gg_max.csv',index=False)
    temp_min_norm.to_csv('s3://'+ my_bucket +'/'+path_to_output+'min_d_'+tipo+'/'+file_name_out+'/min_d'+tipo+'.csv',index=False)
    idrun_min = path_to_output+'min_d_prov/'+file_name_out+'/min_d_'+tipo+'.csv'
    metadatati_min = pd.DataFrame(data={'CATEGORIA':['TEMPERATURA_NORM']*4,'FLUSSO':['MIN_D_'+tipo.upper()]*4,'TIMESTAMP':[now]*4,'ID':['TEMPERATURA_NORM_MIN_D_'+tipo.upper()+'_'+now]*4,'PATH':[idrun_min]*4,'NOME_PARAMETRO':['INIZIO_INIZIO','INIZIO_FINE','FINESTRA_MEDIA_MOBILE_G','PATH_TO_OUTPUT'],'VALORE_PARAMETRO':[inizio,fine,finestra_media_mobile_g,path_to_output]})
    metadatati_min.to_csv('s3://'+ my_bucket +'/metadati/sistema/temperatura_norm/zeus/metadati/min_d'+tipo+'/'+file_name_out+'/metadati.csv',index=False)
    #metadatati_min.to_csv('./metadati/metadati_prov_gg_min.csv',index=False)

    temp_mean_norm.to_csv('s3://'+ my_bucket +'/'+path_to_output+'mean_d_'+tipo+'/'+file_name_out+'/mean_d_'+tipo+'.csv',index=False)
    idrun_mean = path_to_output+'mean_d_prov/'+file_name_out+'/mean_d_'+tipo+'.csv'
    metadatati_mean = pd.DataFrame(data={'CATEGORIA':['TEMPERATURA_NORM']*4,'FLUSSO':['MEAN_D_'+tipo.upper()]*4,'TIMESTAMP':[now]*4,'ID':['TEMPERATURA_NORM_MEAN_D_'+tipo.upper()+'_'+now]*4,'PATH':[idrun_mean]*4,'NOME_PARAMETRO':['INIZIO_INIZIO','INIZIO_FINE','FINESTRA_MEDIA_MOBILE_G','PATH_TO_OUTPUT'],'VALORE_PARAMETRO':[inizio,fine,finestra_media_mobile_g,path_to_output]})
    #metadatati_mean.to_csv('./metadati/metadati_prov_gg_mean.csv',index=False)
    metadatati_mean.to_csv('s3://'+ my_bucket +'/metadati/sistema/temperatura_norm/zeus/metadati/mean_d_'+tipo+'/'+file_name_out+'/metadati.csv',index=False)
#         temp_max_norm.to_csv('prova_gg_max.csv',index=False)
#         temp_min_norm.to_csv('prova_gg_min.csv',index=False)
#         temp_mean_norm.to_csv('prova_gg_mean.csv',index=False)
    print('\n')
#     else:
#         print('Calcolo osservatorio giornaliero con funzioni armoniche di 6^ grado')
#         temp_oss_sv = calcolo_oss_gg('s3://'+ my_bucket +'/'+file_input,inizio_dt,fine_dt,finestra_media_mobile_g)
#         #temp_oss_sv.drop(columns='dayofyear',inplace=True,axis=1)
        
#         temp_oss_sv.to_csv('s3://'+ my_bucket +'/'+path_to_output+'mean_d_'+tipo+'/'+file_name_out+'/mean_d_'+tipo+'.csv',index=False)
#         idrun = path_to_output+'mean_d_oss/'+file_name_out+'/mean_d_'+tipo+'.csv'
#         metadati = pd.DataFrame(data={'CATEGORIA':['TEMPERATURA_NORM']*4,'FLUSSO':['MEAN_D_'+tipo.upper()]*4,'TIMESTAMP':[now]*4,'ID':['TEMPERATURA_NORM_MEAN_D_'+tipo.upper()+'_'+now]*4,'PATH':[idrun]*4,'NOME_PARAMETRO':['INIZIO','FINE','FINESTRA_MEDIA_MOBILE_G','PATH_TO_OUTPUT'],'VALORE_PARAMETRO':[inizio,fine,finestra_media_mobile_g,path_to_output]})
#         metadati.to_csv('s3://'+ my_bucket +'/metadati/sistema/zeus/temperatura_norm/zeus/metadati/mean_d_'+tipo+'oss/'+file_name_out+'/metadati.csv',index=False)
#         #metadati.to_csv('./metadati/metadati_oss_gg_max.csv',index=False)

#         #         temp_oss_sv.to_csv('prova_gg_oss.csv',index=False)
#         print('\n')
    print("Procedura terminata.")
    
#     return temp_max_norm, temp_min_norm, temp_mean_norm, temp_oss_sv

In [44]:
#main_gg('preprocessato/sistema/temperatura_norm/zeus/storico/T_x_OSS_new.csv','2018-01-01','2020-12-31',15,'./')
main_gg('preprocessato/sistema/temperatura/epson/temperatura','2000-01-01','2020-12-31',15,'epson_centralizzato/','prov')
##### NUOVO PATH: 'preprocessato/sistema/temperatura/epson/temperatura'

Funzione dedicata al calcolo delle temperature giornaliere e mensili.


Calcolo temperatura normale giornaliera per prov con funzioni armoniche di 6^ grado
Lettura file temperature per PROVINCIA


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Numero provincia presenti:  102
Calcolo deviazione standard.
Calcolo media giornaliera.
Inizio procedura di fit con armoniche di sesto grado.
Conversione in gradi giorno.
Calcolo percentili per curve di minimo, massimo e media.
Calcolo intervalli di confidenza per curve di minimo, massimo e media.


Procedura terminata.


In [41]:
### FILE pre-centralizzazione epson
old = pd.read_csv('s3://zus-prod-s3/./mean_d_oss/20000101_20201231_20210510074316/mean_d_oss.csv')
old

,dayofyear,mese,giorno,codice_oss,gradi_giorno_yhat,gradi_giorno_T_oss,T_oss,std,var,yhat,percentile_10_yhat,percentile_90_yhat,percentile_10_T_oss,percentile_90_T_oss,c_i_95_T_oss_l,c_i_95_T_oss_h,c_i_95_yhat_l,c_i_95_yhat_h
0,1,1,1,11,14.849695,15.487500,2.512500,2.746581,7.543705,3.150305,-0.370811,6.671421,-1.008616,6.033616,2.209186,2.815814,2.846990,3.453619
1,1,1,1,13,14.262341,15.262500,2.737500,2.591916,6.718030,3.737659,0.414823,7.060496,-0.585337,6.060337,2.451266,3.023734,3.451425,4.023894
2,1,1,1,14,16.307430,17.275000,0.725000,2.819506,7.949612,1.692570,-1.922036,5.307176,-2.889606,4.339606,0.413632,1.036368,1.381202,2.003938
3,1,1,1,15,13.618532,13.962500,4.037500,2.676694,7.164693,4.381468,0.949946,7.812990,0.605978,7.469022,3.741903,4.333097,4.085872,4.677065
4,1,1,1,16,11.077590,10.725000,7.275000,3.039290,9.237281,6.922410,3.026041,10.818780,3.378631,11.171369,6.939361,7.610639,6.586771,7.258050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6583,366,12,31,25,8.839712,10.166667,7.833333,2.986321,8.918112,9.160288,5.331825,12.988752,4.004870,11.661797,7.503544,8.163123,8.830499,9.490078
6584,366,12,31,26,8.243516,8.875000,9.125000,3.145037,9.891260,9.756484,5.724546,13.788422,5.093062,13.156938,8.777683,9.472317,9.409167,10.103802
6585,366,12,31,27,12.737714,14.141667,3.858333,3.438749,11.824998,5.262286,0.853810,9.670763,-0.550143,8.266810,3.478580,4.238086,4.882533,5.642040
6586,366,12,31,28,5.406048,6.541667,11.458333,2.631762,6.926171,12.593952,9.220033,15.967870,8.084415,14.832252,11.167699,11.748968,12.303317,12.884586


In [43]:
### Filo post-centralizzazione epson
new = pd.read_csv('s3://zus-prod-s3/epson_centralizzato/mean_d_oss/20000101_20201231_20210510144835/mean_d_oss.csv')
new

,dayofyear,mese,giorno,osservatorio,T_mean_m,T_min_abs,T_max_abs,gradi_giorno_yhat_mean,gradi_giorno_T_mean,std,var,yhat_mean,percentile_mean_10_T_norm,percentile_mean_90_T_norm,percentile_mean_10_yhat,percentile_mean_90_yhat,c_i_95_T_mean_m_l,c_i_95_T_mean_m_h,c_i_95_yhat_mean_l,c_i_95_yhat_mean_h
0,1,1,1,11.0,3.687500,-5.0,12.0,13.837026,14.312500,2.446740,5.986536,4.162974,0.550779,6.824221,1.026253,7.299695,3.274760,4.100240,3.750234,4.575714
1,1,1,1,13.0,3.250000,-3.0,12.0,13.694006,14.750000,2.478822,6.144558,4.305994,0.072150,6.427850,1.128144,7.483843,2.831848,3.668152,3.887841,4.724146
2,1,1,1,14.0,2.312500,-6.0,14.0,15.184100,15.687500,2.671472,7.136763,2.815900,-1.112327,5.737327,-0.608927,6.240727,1.861850,2.763150,2.365250,3.266551
3,1,1,1,15.0,3.875000,-5.0,12.0,13.074372,14.125000,2.604532,6.783588,4.925628,0.535990,7.214010,1.586618,8.264638,3.435642,4.314358,4.486270,5.364986
4,1,1,1,16.0,6.875000,-1.0,16.0,10.702884,11.125000,2.779399,7.725057,7.297116,3.311811,10.438189,3.733927,10.860305,6.406143,7.343857,6.828260,7.765973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6583,366,12,31,25.0,7.500000,-2.0,15.0,8.717160,10.500000,2.693736,7.256216,9.282840,4.046630,10.953370,5.829470,12.736210,7.045594,7.954406,8.828433,9.737246
6584,366,12,31,26.0,8.777778,-2.0,16.0,8.000788,9.222222,2.743894,7.528954,9.999212,5.260106,12.295450,6.481540,13.516884,8.314910,9.240645,9.536344,10.462079
6585,366,12,31,27.0,2.722222,-7.0,11.0,13.193074,15.277778,3.466396,12.015903,4.806926,-1.721698,7.166142,0.363006,9.250846,2.137476,3.306968,4.222180,5.391672
6586,366,12,31,28.0,10.500000,0.0,17.0,5.801564,7.500000,2.697985,7.279125,12.198436,7.041183,13.958817,8.739619,15.657253,10.044877,10.955123,11.743313,12.653559


In [ ]:
# import pandas as pd
# path_to_output = 'preprocessato/utente/temperatura_norm/zeus/'
# SENZA######################
df = pd.read_csv('s3://zus-prod-s3/./mean_d_oss/best/mean_d_oss_20000101_20201231.csv')
df#[(df['mese']==1) & (df['provincia']==1)].iloc[0:5]# & (df['giorno']==1)]
# df.to_csv('gg_prov_test.csv',index=False)

In [ ]:
# import pandas as pd
# path_to_output = 'preprocessato/utente/temperatura_norm/zeus/'
# CON ######################
df = pd.read_csv('s3://zus-prod-s3/preprocessato/utente/temperatura_norm/zeus/mean_d_oss/best/mean_d_oss_20000101_20201231.csv')
df#[(df['mese']==1) & (df['provincia']==1)].iloc[0:5]# & (df['giorno']==1)]
# df.to_csv('gg_prov_test.csv',index=False)

In [ ]:
df = pd.read_csv('s3://zus-prod-s3/preprocessato/aleedo/temperatura_norm/zeus/mean_d_prov/best/mean_d_prov_20000101_20201231.csv')
df[(df['dayofyear']>=93) & (df['dayofyear']<=96) & (df['provincia']==1)]

In [ ]:
# import pandas as pd
# path_to_output = 'preprocessato/utente/temperatura_norm/zeus/'
df = pd.read_csv('s3://zus-prod-s3/preprocessato/utente/temperatura_norm/zeus/mean_d_prov/best/mean_d_prov_20000101_20201231.csv')
df[(df['monthday']=="01-01")]
# df.to_csv('gg_prov_test.csv',index=False)

In [ ]:
df = pd.read_csv('s3://zus-prod-s3/preprocessato/utente/temperatura_norm/zeus/mean_d_oss/best/mean_d_oss_20000101_20201231.csv')
df

In [ ]:
loro = pd.read_csv('s3://zus-prod-s3/preprocessato/zeu_user/temperatura_norm/zeus/min_d_prov/best/min_d_prov_20090101_20201231.csv')
loro

In [ ]:
import pandas as pd
import numpy as np
df_prod = pd.read_csv('s3://zus-prod-s3/output/sistema/modello_industriali_diretti/zeus/logs/20200101_20201231_20210401114002/cabine_temperature_incomplete.csv')
df_prod.drop_duplicates(subset=['year','CABINE_TEMPERATURE_INCOMPLETE'],inplace=True)
df_prod.drop('MESE_ANNO',axis=1,inplace=True)
df_prod['year'] = df_prod.apply(lambda x: [2015,2016,2017,2018,2019] if np.isnan(x['year']) else x['year'],axis=1)
df_prod.explode('year').iloc[0:60]

In [ ]:
df_old = pd.read_csv('s3://zus-prod-s3/preprocessato/utente/temperatura_norm/zeus/mean_d_prov/best/mean_d_prov_20000101_20201231.csv')
df_old[df_old['monthday']=="01-01"]

In [ ]:
old = pd.read_csv('df_con_92.csv')
# old
old['date'] = pd.to_datetime(old['date'])
max(old['date'])
# old[(old['date'].dt.month==9) & (old['date'].dt.day==30)]

In [8]:
epson = pd.read_csv('s3://zus-prod-s3/preprocessato/sistema/temperatura/epson/temperatura/202012/epson_best.csv')
epson

,AREA,CAPOLUOGO,PROVINCIA,CIELO,COD_ISTAT,FENOMENO,DATA,DATA_VAL,TIPO,LATITUDINE,LONGITUDINE,TEMPERATURA_MAX,TEMPERATURA_MIN,UDM,QUOTA,REGIONE,DATA_AGGIORNAMENTO
0,NaN,TORINO,1,NaN,1272,NaN,2020-12-02,2020-12-01,1,45.06,7.68,5.0,1.0,°C,239,NaN,2021-05-05 13:44:19
1,NaN,VERCELLI,2,NaN,2158,NaN,2020-12-02,2020-12-01,1,45.32,8.42,6.0,1.0,°C,130,NaN,2021-05-05 13:44:19
2,NaN,NOVARA,3,NaN,3106,NaN,2020-12-02,2020-12-01,1,45.44,8.61,6.0,1.0,°C,162,NaN,2021-05-05 13:44:19
3,NaN,CUNEO,4,NaN,4078,NaN,2020-12-02,2020-12-01,1,44.49,7.54,4.0,0.0,°C,534,NaN,2021-05-05 13:44:19
4,NaN,ASTI,5,NaN,5005,NaN,2020-12-02,2020-12-01,1,44.89,8.20,6.0,1.0,°C,123,NaN,2021-05-05 13:44:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3157,NaN,VIBO VALENTIA,102,NaN,102047,NaN,2021-01-01,2020-12-31,1,38.67,16.10,10.0,5.0,°C,476,NaN,2021-05-05 13:44:19
3158,NaN,VERBANIA,103,NaN,103072,NaN,2021-01-01,2020-12-31,1,45.92,8.54,5.0,2.0,°C,197,NaN,2021-05-05 13:44:19
3159,NaN,MONZA,108,NaN,108033,NaN,2021-01-01,2020-12-31,1,45.58,9.27,6.0,1.0,°C,162,NaN,2021-05-05 13:44:19
3160,NaN,FERMO,109,NaN,109006,NaN,2021-01-01,2020-12-31,1,43.16,13.71,8.0,2.0,°C,319,NaN,2021-05-05 13:44:19


In [ ]:
fit_harmonics(np.array([1,1,1,2,2,2,2,2,7]),6)

In [ ]:
fit_harmonics(np.array([2,2,2,2,2,1,1,1,7]),6)

In [ ]:
len(np.array([1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,10]))

In [ ]:
d = pd.DataFrame({'temp':[ 2.3,  2.25, 2.5,  3.35, 4.,   3.6,
  4.,   3.05, 3.2,  4.35, 3.9,  4.5,
  2.65, 2.75, 2.2,  1.4,  1.75, 2.6,
  3.85, 3.55, 3.25, 3.7,  2.7,  2.6,
  3.1,  2.5,  3.,   4.2,  2.85, 2.3,
  2.65, 3.35, 3.05, 3.3,  3.,   2.2,
  3.5,  2.85, 2.85, 4.3,  3.55, 3.1,
  2.95, 3.35, 3.55, 4.6,  4.75, 5.1,
  4.1,  5.15, 5.55, 4.95, 5.45, 6.25,
  6.15, 6.8,  6.7,  7.5,  5.8, 12.25,
  7.55, 9.3,  7.95, 7.9,  6.45, 7.25,
  6.55, 7.05, 7.6,  7.1,  8.85, 8.6 ,
  9.4,  9.05, 9.65, 9.9,  9.8, 10.65,
 10.7,  8.85, 9.05, 9.6, 10.25, 9.6 ,
  9.35, 9.7, 10.3, 10.8, 12.05,11.6 ,
 12.9, 11.5, 12.3, 12.15,10.8, 11.7 ,
 12.55,13.25,12.05,12.95,12.95,12.15,
 13.15,13.75,14.,  13.3, 13.5, 13.05,
 12.9, 12.2, 13.4, 14.4, 15.6, 15.4 ,
 15.85,15.85,15.45,14.35,14.65,14.8 ,
 13.7, 15.1, 14.95,15.85,15.45,15.5 ,
 15.35,16.85,17.25,17.45,17.1, 18.25,
 18.15,16.8, 16.95,14.95,16.1, 17.5 ,
 17.,  17.35,17.45,17.55,18.6, 19.8 ,
 20.05,18.95,18.05,18.3, 17.65,18.3 ,
 18.05,16.95,17.22222222, 18.8, 19.7, 19.95,
 20.1, 20.,  19.85,20.45,20.2, 20.6 ,
 21.65,21.2, 21.05,21.05,20.55,20.95,
 20.7, 21.65,22.,  21.3, 22.1, 22.2 ,
 22.2, 22.6, 22.8, 23.1, 23.2, 23.6 ,
 22.45,24.1, 23.3, 22.85,23.2, 23.2 ,
 24.05,23.35,22.85,22.85,22.75,23.  ,
 23.9, 23.75,23.05,23.8, 24.7, 24.45,
 23.05,23.65,23.45,23.9, 23.9, 23.7 ,
 23.1, 23.15,23.6, 23.65,24.3, 24.3 ,
 22.8, 23.35,23.95,23.,  23.3, 22.8 ,
 23.8, 23.75,23.7, 23.6, 22.6, 21.85,
 22.6, 23.15,22.65,22.6, 21.45,21.1 ,
 21.7, 22.85,23.35,22.9, 23.05,23.15,
 23.4, 22.05,21.75,21.65,21.5, 21.95,
 21.95,21.65,21.25,20.75,20.5, 21.35,
 21.1, 20.75,20.55,20.85,20.1, 20.85,
 20.85,20.6, 20.6, 19.45,19.05,18.45,
 18.2, 18.65,18.8, 17.5, 17.65,18.05,
 18.2, 18.45,17.85,17.15,17.25,17.1 ,
 16.95,17.05,16.65,16.85,17.6, 17.5 ,
 16.1, 16.1, 16.25,17.05,15.85,15.4 ,
 15.5, 14.65,14.95,14.75,14.6, 13.3 ,
 12.45,13.55,12.85,12.3, 12.25,12.45,
 12.7, 12.1, 11.75,12.95,12.15,13.2 ,
 12.25,11.7, 11.8, 11.7,  9.95,10.7 ,
 10.7, 10.7,  9.85,10.9,  9.85,10.3 ,
 10.85, 9.25, 9.2,  9.1,  9.45,10.2 ,
  9.,   8.8,  8.5,  8.,   6.6,  7.25,
  7.25, 7.15, 5.95, 6.1,  6.4,  6.45,
  5.55, 5.1,  4.85, 5.65, 5.15, 5.1 ,
  5.1,  4.65, 5.05, 5.35, 4.95, 4.8 ,
  4.65, 4.15, 4.,   4.4,  4.05, 3.75,
  3.15, 2.65, 3.,   2.4,  2.3,  1.65,
  1.25, 2.6,  2.35, 3.55, 3.65, 4.2,
  1.75, 1.,   1.4,  2.,   1.7,  2.45]})

In [ ]:
con = pd.read_csv('temp_norm_con_92.csv')
senza = pd.read_csv('temp_norm_senza_92.csv')
# senza
con = con[con['provincia']==1]
senza = senza[senza['provincia']==1]

In [ ]:
ycon = con['T_mean_m'].values.ravel()
ysenza = senza['T_mean_m'].values.ravel()

In [ ]:
yhat_mean_con = fit_harmonics(ycon, 6)
yhat_mean_senza = fit_harmonics(ysenza, 6)

In [ ]:
yhat_mean_con[92:95]

In [13]:
epson_best = pd.read_csv('s3://zus-prod-s3/preprocessato/sistema/temperatura/epson/temperatura/202012/epson_best.csv')
epson_best['OSSERVATORIO'].nunique()

18

In [ ]:
array([ 2.50600773,  2.48847208,  2.47466042,  2.46448031,  2.45784016,
        2.45464994,  2.45482175,  2.45827047,  2.46491431,  2.47467532,
        2.48747994,  2.5032594 ,  2.52195011,  2.54349408,  2.56783912,
        2.59493917,  2.62475443,  2.65725151,  2.69240348,  2.73018991,
        2.77059676,  2.81361635,  2.85924711,  2.90749339,  2.95836521,
        3.01187784,  3.06805149,  3.12691085,  3.18848457,  3.2528048 ,
        3.31990659,  3.38982731,  3.46260602,  3.53828284,  3.61689824,
        3.69849242,  3.78310458,  3.87077221,  3.96153046,  4.05541137,
        4.15244326,  4.25265004,  4.35605059,  4.46265812,  4.57247964,
        4.68551536,  4.80175826,  4.92119355,  5.04379836,  5.16954129,
        5.29838218,  5.43027186,  5.56515192,  5.70295468,  5.84360304,
        5.98701058,  6.13308158,  6.28171118,  6.43278562,  6.58618247,
        6.74177098,  6.89941253,  7.05896103,  7.22026345,  7.38316044,
        7.5474869 ,  7.71307269,  7.87974332,  8.04732073,  8.21562407,
        8.38447051,  8.55367611,  8.72305665,  8.89242856,  9.06160974,
        9.23042049,  9.39868439,  9.56622919,  9.73288761,  9.89849827,
       10.06290643, 10.2259648 , 10.38753432, 10.54748479, 10.7056956 ,
       10.86205632, 11.0164672 , 11.16883977, 11.31909716, 11.46717455,
       11.61301942, 11.75659184, 11.89786454, 12.03682309, 12.17346586,
       12.30780394, 12.43986109, 12.56967344, 12.69728926, 12.82276861,
       12.94618291, 13.06761445, 13.18715588, 13.30490954, 13.42098688,
       13.53550766, 13.64859925, 13.76039582, 13.87103744, 13.98066929,
       14.08944068, 14.19750417, 14.30501461, 14.41212819, 14.51900147,
       14.62579045, 14.73264957, 14.83973084, 14.94718283, 15.05514987,
       15.16377109, 15.27317964, 15.38350186, 15.49485654, 15.60735423,
       15.72109652, 15.83617551, 15.95267321, 16.0706611 , 16.19019966,
       16.31133808, 16.43411389, 16.5585528 , 16.68466853, 16.81246274,
       16.94192496, 17.07303271, 17.2057516 , 17.3400355 , 17.4758268 ,
       17.6130567 , 17.75164565, 17.89150367, 18.03253092, 18.17461817,
       18.3176474 , 18.4614924 , 18.60601944, 18.7510879 , 18.89655105,
       19.04225671, 19.18804804, 19.3337643 , 19.47924161, 19.6243137 ,
       19.76881271, 19.91256993, 20.05541659, 20.19718451, 20.33770692,
       20.47681905, 20.61435887, 20.75016765, 20.88409061, 21.01597744,
       21.1456828 , 21.27306684, 21.39799557, 21.52034128, 21.63998281,
       21.75680589, 21.87070333, 21.9815752 , 22.08932895, 22.19387953,
       22.29514934, 22.3930683 , 22.48757369, 22.57861011, 22.66612931,
       22.75008998, 22.83045752, 22.90720381, 22.98030686, 23.04975055,
       23.11552423, 23.17762237, 23.23604419, 23.29079324, 23.341877  ,
       23.38930646, 23.43309572, 23.47326154, 23.50982295, 23.54280085,
       23.57221762, 23.59809671, 23.62046232, 23.63933903, 23.65475151,
       23.66672419, 23.67528105, 23.68044531, 23.68223928, 23.68068417,
       23.67579992, 23.66760514, 23.65611698, 23.64135111, 23.62332169,
       23.60204142, 23.57752157, 23.54977202, 23.51880146, 23.48461742,
       23.44722649, 23.40663449, 23.36284667, 23.31586789, 23.26570291,
       23.21235659, 23.15583418, 23.09614153, 23.03328539, 22.96727368,
       22.89811571, 22.82582245, 22.7504068 , 22.67188378, 22.5902708 ,
       22.50558782, 22.41785754, 22.32710558, 22.2333606 , 22.13665444,
       22.03702217, 21.93450218, 21.82913619, 21.72096926, 21.61004974,
       21.49642926, 21.38016256, 21.26130743, 21.13992455, 21.01607727,
       20.88983145, 20.76125519, 20.63041858, 20.49739345, 20.36225301,
       20.22507158, 20.08592424, 19.94488646, 19.80203379, 19.65744145,
       19.51118397, 19.36333486, 19.21396619, 19.06314827, 18.91094928,
       18.75743494, 18.60266819, 18.44670887, 18.28961346, 18.1314348 ,
       17.97222189, 17.81201964, 17.65086876, 17.48880557, 17.32586192,
       17.16206516, 16.99743805, 16.83199882, 16.66576123, 16.49873461,
       16.33092405, 16.16233057, 15.99295128, 15.82277971, 15.65180603,
       15.4800174 , 15.30739836, 15.13393115, 14.9595962 , 14.78437253,
       14.60823822, 14.43117092, 14.25314835, 14.07414879, 13.89415165,
       13.71313798, 13.531091  , 13.34799666, 13.16384414, 12.97862639,
       12.79234063, 12.6049888 , 12.41657811, 12.22712137, 12.0366375 ,
       11.84515183, 11.6526965 , 11.45931073, 11.26504108, 11.06994169,
       10.87407443, 10.67750903, 10.48032318, 10.28260248, 10.08444049,
        9.88593861,  9.68720594,  9.48835912,  9.28952205,  9.09082564,
        8.89240744,  8.69441128,  8.4969868 ,  8.30028902,  8.10447779,
        7.90971722,  7.71617512,  7.52402239,  7.33343231,  7.14457994,
        6.95764137,  6.77279305,  6.59021106,  6.41007039,  6.23254422,
        6.05780318,  5.88601468,  5.71734217,  5.55194448,  5.38997518,
        5.2315819 ,  5.07690576,  4.92608082,  4.77923351,  4.63648219,
        4.49793668,  4.36369785,  4.23385735,  4.10849724,  3.98768982,
        3.8714974 ,  3.75997226,  3.6531565 ,  3.5510821 ,  3.45377097,
        3.36123507,  3.27347656,  3.19048807,  3.11225296,  3.03874568,
        2.96993215,  2.90577021,  2.84621007,  2.79119491,  2.74066134,
        2.6945401 ,  2.65275659,  2.6152316 ,  2.58188193,  2.5526211 ,
        2.52736002])
